# Inhaltsbasiertes Empfehlungssystem für Netflix-Filme

**Gruppe 1:** <br>
Theen, Johannes (TH München)<br>
Utz, Elisabeth (OTH Amberg-Weiden)<br>
Yaruchyk, Oleg (TH München)

## 1. Import von Bibliotheken, Klassen und Funktionen

In [1]:
import pandas as pd
import numpy as np
import string
# Vektorisierung
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
# Dimensionsreduktion
from scipy import sparse
from sklearn.preprocessing import MaxAbsScaler
from sklearn.decomposition import TruncatedSVD
# Ähnlichkeitsberechnung
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import polynomial_kernel
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.pairwise import laplacian_kernel

## 2. Einlesen und Analyse der Datenbank
Die Datenbank besteht aus 12 Spalten und 6.234 Zeilen. Anhand der Spalte "director" ist bereits zu erkennen, dass manche Zellen den Wert "NaN" enthalten. Die Zeile "count" in der anschließenden Ansicht macht noch einmal deutlich, dass nicht alle Spalten über 6.234 Einträge verfügen.

In [2]:
raw_data = pd.read_csv('netflix_titles.csv')
print(raw_data.shape)
raw_data.head()

(6234, 12)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [3]:
raw_data.describe(include='all')

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
count,6.234000e+03,6234,6234,4265,5664,5758,6223,6234.00000,6224,6234,6234,6234
unique,NaN,2,6172,3301,5469,554,1524,NaN,14,201,461,6226
top,NaN,Movie,The Silence,"Raúl Campos, Jan Suter",David Attenborough,United States,"January 1, 2020",NaN,TV-MA,1 Season,Documentaries,A surly septuagenarian gets another chance at ...
freq,NaN,4265,3,18,18,2032,122,NaN,2027,1321,299,3
mean,7.670368e+07,NaN,NaN,NaN,NaN,NaN,NaN,2013.35932,NaN,NaN,NaN,NaN
std,1.094296e+07,NaN,NaN,NaN,NaN,NaN,NaN,8.81162,NaN,NaN,NaN,NaN
min,2.477470e+05,NaN,NaN,NaN,NaN,NaN,NaN,1925.00000,NaN,NaN,NaN,NaN
25%,8.003580e+07,NaN,NaN,NaN,NaN,NaN,NaN,2013.00000,NaN,NaN,NaN,NaN
50%,8.016337e+07,NaN,NaN,NaN,NaN,NaN,NaN,2016.00000,NaN,NaN,NaN,NaN
75%,8.024489e+07,NaN,NaN,NaN,NaN,NaN,NaN,2018.00000,NaN,NaN,NaN,NaN


## 3. Preprocessing

Daher werden im nächsten Schritt alle Einträge "NaN" durch eine leere Zelle ersetzt. Anschließend zeigt die Zeile **count** in jeder Spalte 6.234 Eintäge.

In [4]:
raw_data = raw_data.fillna('')
raw_data.describe(include='all')

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
count,6.234000e+03,6234,6234,6234,6234,6234,6234,6234.00000,6234,6234,6234,6234
unique,NaN,2,6172,3302,5470,555,1525,NaN,15,201,461,6226
top,NaN,Movie,The Silence,,,United States,"January 1, 2020",NaN,TV-MA,1 Season,Documentaries,A surly septuagenarian gets another chance at ...
freq,NaN,4265,3,1969,570,2032,122,NaN,2027,1321,299,3
mean,7.670368e+07,NaN,NaN,NaN,NaN,NaN,NaN,2013.35932,NaN,NaN,NaN,NaN
std,1.094296e+07,NaN,NaN,NaN,NaN,NaN,NaN,8.81162,NaN,NaN,NaN,NaN
min,2.477470e+05,NaN,NaN,NaN,NaN,NaN,NaN,1925.00000,NaN,NaN,NaN,NaN
25%,8.003580e+07,NaN,NaN,NaN,NaN,NaN,NaN,2013.00000,NaN,NaN,NaN,NaN
50%,8.016337e+07,NaN,NaN,NaN,NaN,NaN,NaN,2016.00000,NaN,NaN,NaN,NaN
75%,8.024489e+07,NaN,NaN,NaN,NaN,NaN,NaN,2018.00000,NaN,NaN,NaN,NaN


Damit SchauspielerInnen und RegisseurInnen in die Berechnungen einbezogen werden können, werden die Vor- und Nachnamen zusammen- und alles klein geschrieben (z.B. wird Richard Finn zu richardfinn), sodass z.B. die Schauspielerinnen Kate Winslet und Kate Hudson nicht allein aufgrund ihres Vornamens zu einer scheinbaren erhöhten Ähnlichkeit führen. Auch aus mehreren Wörtern bestehende Ländernamen und Kategorien werden auf diese Weise verändert.

In [5]:
def organize_data(data):
    # function to wirte connected names in one word
    data = data.str.replace(' ','')
    data = data.str.lower()
    data = data.str.replace(',',', ')
    return data

raw_data['type'] = organize_data(raw_data['type'])
raw_data['director'] = organize_data(raw_data['director'])
raw_data['cast'] = organize_data(raw_data['cast'])
raw_data['country'] = organize_data(raw_data['country'])
raw_data['listed_in'] = organize_data(raw_data['listed_in'])

raw_data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,movie,Norm of the North: King Sized Adventure,"richardfinn, timmaltby","alanmarriott, andrewtoth, briandobson, colehow...","unitedstates, india, southkorea, china","September 9, 2019",2019,TV-PG,90 min,"children&familymovies, comedies",Before planning an awesome wedding for his gra...
1,80117401,movie,Jandino: Whatever it Takes,,jandinoasporaat,unitedkingdom,"September 9, 2016",2016,TV-MA,94 min,stand-upcomedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,tvshow,Transformers Prime,,"petercullen, sumaleemontano, frankwelker, jeff...",unitedstates,"September 8, 2018",2013,TV-Y7-FV,1 Season,kids'tv,"With the help of three human allies, the Autob..."
3,80058654,tvshow,Transformers: Robots in Disguise,,"willfriedle, darrencriss, constancezimmer, kha...",unitedstates,"September 8, 2018",2016,TV-Y7,1 Season,kids'tv,When a prison ship crash unleashes hundreds of...
4,80125979,movie,#realityhigh,fernandolebrija,"nestacooper, katewalsh, johnmichaelhiggins, ke...",unitedstates,"September 8, 2017",2017,TV-14,99 min,comedies,When nerdy high schooler Dani finally attracts...


Im folgenden Schritt werden noch enthaltene (Satz-)Zeichen wie z.B. "&" in den Spalten "title_pp", "director", "cast", "country", "rating" und "listed_in" entfernt. Für die Titel wird dabei eine zusätzliche Spalte (title_pp für preprocess) angelegt, da das Empfehlungsprogramm am Ende auf die unveränderten Titel zurückgreifen können muss.

In [6]:
# delete punctuation
raw_data['title_pp'] = [row.translate(str.maketrans("","", string.punctuation)) for row in raw_data['title']]
raw_data['cast'] = [row.translate(str.maketrans("","", string.punctuation)) for row in raw_data['cast']]
raw_data['listed_in'] = [row.translate(str.maketrans("","", string.punctuation)) for row in raw_data['listed_in']]
raw_data['director'] = [row.translate(str.maketrans("","", string.punctuation)) for row in raw_data['director']]
raw_data['country'] = [row.translate(str.maketrans("","", string.punctuation)) for row in raw_data['country']]
raw_data['rating'] = [row.translate(str.maketrans("","", string.punctuation)) for row in raw_data['rating']]
raw_data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,title_pp
0,81145628,movie,Norm of the North: King Sized Adventure,richardfinn timmaltby,alanmarriott andrewtoth briandobson colehoward...,unitedstates india southkorea china,"September 9, 2019",2019,TVPG,90 min,childrenfamilymovies comedies,Before planning an awesome wedding for his gra...,Norm of the North King Sized Adventure
1,80117401,movie,Jandino: Whatever it Takes,,jandinoasporaat,unitedkingdom,"September 9, 2016",2016,TVMA,94 min,standupcomedy,Jandino Asporaat riffs on the challenges of ra...,Jandino Whatever it Takes
2,70234439,tvshow,Transformers Prime,,petercullen sumaleemontano frankwelker jeffrey...,unitedstates,"September 8, 2018",2013,TVY7FV,1 Season,kidstv,"With the help of three human allies, the Autob...",Transformers Prime
3,80058654,tvshow,Transformers: Robots in Disguise,,willfriedle darrencriss constancezimmer kharyp...,unitedstates,"September 8, 2018",2016,TVY7,1 Season,kidstv,When a prison ship crash unleashes hundreds of...,Transformers Robots in Disguise
4,80125979,movie,#realityhigh,fernandolebrija,nestacooper katewalsh johnmichaelhiggins keith...,unitedstates,"September 8, 2017",2017,TV14,99 min,comedies,When nerdy high schooler Dani finally attracts...,realityhigh


## 4. Vektorisierung des Dokuments

Damit die Daten ausgewertet und verglichen werden können, werden sie nun mithilfe der Klasse CountVectorizer von scikit-learn in Vektoren transformiert. Es wird davon ausgegangen, dass die Netflix-ID, das Erscheinungsjahr, das Datum, an dem der Eintrag Netflix hinzugefügt wurde sowie die Länge des Films keinen (nennenswerten) Einfluss auf das Empfehlungssystem haben, weshalb diese Kategorien schon im Voraus ausgeschlossen werden, um das Datenvolumen zu reduzieren.<br> Auf die Spalte "description" wird der TF-IDF-Vectorizer von scikit-learn angewandt. So wird zwar nach wie vor die Häufigkeit von Wörtern innerhalb eines Dokuments betrachtet. Allerdings werden Wörter, die in (fast) jedem Dokument vorkommen, weniger stark gewichtet, da sie für uns keinen Mehrwert enthalten. Zusätzlich wurde hier der Parameter "stop_words" aktiviert. Dadurch werden alle Wörter, die sich in der stopwords-Liste des Natural Language Toolkit befinden, ignoriert. Bei stopwords handelt es sich um Wörter wie "a(n)", "the", "this" etc, die in nahezu jedem Dokument vorkommen (können), aber über keinerlei Informationsgehalt verfügen.


In [7]:
# lowercase, tokenize and vectorize the data; for description: ignore stop words;
count_vectorizer= CountVectorizer()
tfidf_vectorizer = TfidfVectorizer(stop_words=set(stopwords.words("english")))

data_type = count_vectorizer.fit_transform(raw_data['type'])
data_title = count_vectorizer.fit_transform(raw_data['title_pp'])
data_director = count_vectorizer.fit_transform(raw_data['director'])
data_cast = count_vectorizer.fit_transform(raw_data['cast'])
data_country = count_vectorizer.fit_transform(raw_data['country'])
data_rating = count_vectorizer.fit_transform(raw_data['rating'])
data_listed_in = count_vectorizer.fit_transform(raw_data['listed_in'])
data_description = tfidf_vectorizer.fit_transform(raw_data['description'])

# 5. Skalierung und Dimensionsreduktion
Jede der oben generierten Variablen stellt nun eine schwach besetzte Matrix dar, die als Compressed Sparse Row (CSR) vorliegt. Um eine Dimensionsreduktion durchführen zu können, werden im nächsten Schritt alle Matrizen aneinander gereiht, sodass eine Matrix mit 6234 Zeilen entsteht, die alle Vektoren als Spalten enthält.

In [8]:
#merge the data
all_matrix = sparse.hstack((data_type, data_title, data_director, data_cast, data_country, data_rating, data_listed_in, data_description), format='csr') 
all_matrix

<6234x54648 sparse matrix of type '<class 'numpy.float64'>'
	with 190910 stored elements in Compressed Sparse Row format>

Vor der Dimensionsreduktion wird die Matrix skaliert. Da es sich um eine schwach besetzte Matrix handelt, darf durch die Skalierung keine Verschiebung erfolgen. Als Scaler für schwach besetzte Matrizen empfiehlt scikit-learn daher den MaxAbsScaler, bei dem die Matrix so skaliert wird, dass der maximale Absolutwert eines einzelnen Wertes 1 ist.

In [9]:
all_matrix_scale = MaxAbsScaler().fit_transform(all_matrix)
all_matrix_scale

<6234x54648 sparse matrix of type '<class 'numpy.float64'>'
	with 190910 stored elements in Compressed Sparse Row format>

Im nächsten Schritt erfolgt die Dimensionsreduktion. Für schwach besetzte Matrizen wird von scikit-learn TruncatedSVD empfohlen. Reduziert man die Matrix auf 5.100 Komponenten, werden durch diese Komponenten ca. 95% der Varianz der Daten erklärt.

In [15]:
svd = TruncatedSVD(n_components = 5100, n_iter = 5)
all_matrix_svd = svd.fit_transform(all_matrix_scale)
# summarize explained variance ratio for all 5,100 components
explained_variance = svd.explained_variance_ratio_.sum()
print("The explainded variance amounts to " + str(explained_variance))

The explainded variance amounts to 0.9468654574713558


# 6. Ähnlichkeitsanalyse und Empfehlungsfunktion
Anschließend wird die Kosinus-Ähnlichkeit der einzelnen Einträge zueinander berechnet. Um die Auswirkungen von Skalierung und Dimensionsreduktion beurteilen zu können, wird die Ähnlichkeit für die unskalierte, die skalierte und die dimensionsreduzierte Matrix berechnet.

In [16]:
cos_similarity = cosine_similarity(all_matrix, all_matrix)
cos_similarity_scale = cosine_similarity(all_matrix_scale, all_matrix_scale)
cos_similarity_svd = cosine_similarity(all_matrix_svd,all_matrix_svd)

Für die Empfehlungen wird nun eine Funktion erstellt, die die Ergebnisse der Ähnlichkeitsberechnung absteigend sortiert und die zugehörigen Filme der zehn besten Ergebnisse zurückgibt.

In [18]:
indices = pd.Series(raw_data['title'])


def recommendations(title, cosine_sim):
    
    recommended_movies = []
    
    # getting the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indices of the 10 most similar movies
    top_10_indices = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indices:
        recommended_movies.append(list(raw_data['title'])[i])
        
    return recommended_movies

In [19]:
recommendations("Men in Black", cos_similarity)

['Men in Black II',
 'Wild Wild West',
 'Small Soldiers',
 'Austin Powers in Goldmember',
 'Space Cowboys',
 'Skiptrace',
 'Singularity',
 'Dragonheart',
 'Tremors 6: A Cold Day in Hell',
 'Otherhood']

In [20]:
recommendations("Men in Black", cos_similarity_scale)

['Men in Black II',
 'Wild Wild West',
 'Small Soldiers',
 'Get Smart',
 'Space Cowboys',
 'Dragonheart',
 'Skiptrace',
 'The Tuxedo',
 'Singularity',
 'Evolution']

In [21]:
recommendations("Men in Black", cos_similarity_svd)

['Men in Black II',
 'Wild Wild West',
 'Small Soldiers',
 'Get Smart',
 'Space Cowboys',
 'Dragonheart',
 'Skiptrace',
 'Singularity',
 'The Tuxedo',
 'The Space Between Us']

## 6.1 Vergleich unterschiedlicher Ähnlichkeitsberechnungen
Im folgenden wird die Ähnlichkeit nicht über die Kosinus-Ähnlichkeit berechnet sondern mithilfe des linearen Kernel von scikit-learn. Anschließend wird die recommendations-Funktion mit den Ergebnissen dieser Ähnlichkeitsberechnugn aufgerufen. Es wurde auch der polynome Kernel getestet (linearer Kernel = polynomer Kernel mit degree = 1). Da das Ergebnis jedoch dasselbe ist (zumindest bis degree = 5), wird hier aus Gründen der Übersichtlichkeit auf die Darstellung verzichtet.

In [22]:
linear_similarity_svd = linear_kernel(all_matrix_svd,all_matrix_svd)
recommendations("Men in Black", linear_similarity_svd)

['Men in Black II',
 'Wild Wild West',
 'Small Soldiers',
 'Space Cowboys',
 'Get Smart',
 'Where the Money Is',
 'The Bounty Hunter',
 'Ant-Man and the Wasp',
 'Black Panther',
 'Thor: Ragnarok']

Die Ergebnisse der Ähnlichkeitsberechnung mit dem Sigmoid-Kernel decken sich mit denen des linearen Kernel:

In [23]:
sigmoid_similarity_svd = sigmoid_kernel(all_matrix_svd,all_matrix_svd)
recommendations("Men in Black", sigmoid_similarity_svd)

['Men in Black II',
 'Wild Wild West',
 'Small Soldiers',
 'Space Cowboys',
 'Get Smart',
 'Where the Money Is',
 'The Bounty Hunter',
 'Ant-Man and the Wasp',
 'Black Panther',
 'Thor: Ragnarok']

Die Ergebnisse des RBF-Kernel weichen dagegen schon merklich von den übrigen Ähnlichkeitsberechnungen ab. Bezogen auf die unskalierte, dimnesionsal noch nicht reduzierte Matrix stimmen lediglich zwei Filme überein:

In [24]:
rbf_similarity_svd = rbf_kernel(all_matrix_svd,all_matrix_svd)
recommendations("Men in Black", rbf_similarity_svd)

['Men in Black II',
 'Otherhood',
 'Blackfish',
 'Cristina',
 'Unacknowledged',
 'Life 2.0',
 'Alarmoty in the Land of Fire',
 'The Force',
 'AlphaGo',
 'My Own Man']

Die gleichen Ergebnisse liefert eine Ähnlichkeitsberechnung mit dem Laplace-Kernel:

In [25]:
laplace_similarity_svd = rbf_kernel(all_matrix_svd,all_matrix_svd)
recommendations("Men in Black", laplace_similarity_svd)

['Men in Black II',
 'Otherhood',
 'Blackfish',
 'Cristina',
 'Unacknowledged',
 'Life 2.0',
 'Alarmoty in the Land of Fire',
 'The Force',
 'AlphaGo',
 'My Own Man']

## 6.2 Gewichtung einzelner Faktoren
Eine Alternative zu einer großen Matrix, deren Dimension reduziert wird, wären mehrere einzelne Matrizen, die unterschiedlich gewichtet werden können. Als Beispiel werden an dieser Stelle drei Kategorien generiert, nämlich
<br>
- Genre: Zusammenfassung der Kategorien 'type', 'listed_in' und 'rating',
- Team: Zusammenfassung der Kategorien 'director' und 'cast',
- Content: Zusammenfassung der Kategorien 'title' und 'description';


Nach der Vektorisierung wird für jede Kategorie die Ähnlichkeitsberechunng (und zuvor ggf. eine Dimnesionsreduktion) durchgeführt.<br>
Anschließend wird durch Addition der unterschiedlich gewichteten Matrizen eine neue Matrix generiert, auf deren Basis die Empfehlung erfolgt. <br>
*Anmerkung: Da in unserem Fall die Dimensionsreduktion mit Abstand die meiste Rechenzeit in Anspruch nimmt, wird im folgenden Beispiel darauf verzichtet.*

In [26]:
# generate new categories
genre = sparse.hstack((data_type, data_rating, data_listed_in), format='csr')
team = sparse.hstack((data_director, data_cast), format='csr')
content = sparse.hstack((data_title, data_description), format='csr')

In [30]:
# calculate cosine similarity
genre_similarity = cosine_similarity(genre, genre)
team_similarity = cosine_similarity(team, team)
content_similarity = cosine_similarity(content, content)

# determine coefficients
coeff_genre = 1.5
coeff_team = 2
coeff_content = 1

# calculate total similarity 
total_similarity = coeff_genre*genre_similarity + coeff_team*team_similarity + coeff_content*content_similarity

# call recommendations function
recommendations("Men in Black", total_similarity)

['Men in Black II',
 'Wild Wild West',
 'Small Soldiers',
 'Black Panther',
 'Austin Powers in Goldmember',
 'Austin Powers: The Spy Who Shagged Me',
 'Thor: Ragnarok',
 'Ant-Man and the Wasp',
 'Tremors 6: A Cold Day in Hell',
 'Where the Money Is']

# 7. 